Импорт необходимых библиотек

In [47]:
from ultralytics import YOLO
import shutil 
import os
import ruamel.yaml as yaml
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import numpy as np
import sys
import io
from contextlib import redirect_stdout

Распаковка данных

In [13]:
shutil.unpack_archive('./preprocessed_images.zip', './data', 'zip') 

Сбор классов

In [2]:
classes=next(os.walk('./data/train/'))[1]

Создание датасета для Yolo

In [41]:


def create_yolo_yaml(config_dict, output_path='./data.yaml'):
    """
    Creates a YOLO-compatible YAML configuration file.
    
    Args:
        config_dict (dict): Dictionary containing YOLO configuration
        output_path (str): Path where YAML file will be saved
    """
    # Initialize YAML with round-trip preservation
    yaml_config = yaml.YAML()
    yaml_config.indent(mapping=2, sequence=4, offset=2)
    yaml_config.width = float('inf')
    
    # Write configuration to YAML file
    with open(output_path, 'w') as file:
        yaml_config.dump(config_dict, file)

# Define YOLO configuration
config = {
    'train': 'train/',
    'valid': 'val/',
    'test': 'test/',
    'nc': len(classes),
    'names': classes
}

# Create YAML file
create_yolo_yaml(config)

Пайплайн использует YOLO, которая логирует свои действия в ./runs/classify а номер папки можно просмотреть в файле который получаем на выходе при завершении пайплайна, такде на выходи получаем обученную модель, результат валидации с метрикой accuracy.

In [51]:
def pipeline(model_select, **kwargs):
    model=None
    if model_select=='yolo11n':
        model = YOLO("yolo11n-cls.pt")
    elif model_select=="yolo11s":
        model = YOLO('yolo11s-cls.pt')
    elif model_select=='yolo11m':
        model = YOLO('yolo11m-cls.pt')
    else:
        print("Данная модель не поддерживается")
    e=1
    for k,w in kwargs.items():
        if k=="epoch":
            e=w
    f = io.StringIO()
    with redirect_stdout(f):
        model.train(data='./data/', epochs=e,verbose=False)
        metrics = model.val()
    output = f.getvalue()
    now = datetime.now()
    fn = now.strftime("%Y-%m-%d_%H-%M-%S.txt")
    with open(fn, 'w') as file:
        file.write(output)
    return model, metrics, output

In [52]:
pipeline('yolo11n',epchos=1)

Ultralytics 8.3.70 🚀 Python-3.11.2 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=./data/, epochs=1, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /home/c10/Рабочий стол/С10/C10M3/data/train... 25000 images, 0 corrupt: 100%|██████████| 25000/25000 [00:00<?, ?it/s]
val: Scanning /home/c10/Рабочий стол/С10/C10M3/data/val... 2000 images, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=1e-05, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 0 dataloader workers
Logging results to runs/classify/train6
Starting training for 1 epochs...

      Epoch    GPU_mem       loss  Instances       Size



               classes   top1_acc   top5_acc: 100%|██████████| 63/63 [00:11<00:00,  5.33it/s]


                   all      0.002      0.006

1 epochs completed in 0.145 hours.
Optimizer stripped from runs/classify/train6/weights/last.pt, 5.8MB
Optimizer stripped from runs/classify/train6/weights/best.pt, 5.8MB

Validating runs/classify/train6/weights/best.pt...
Ultralytics 8.3.70 🚀 Python-3.11.2 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
YOLO11n-cls summary (fused): 112 layers, 2,807,024 parameters, 0 gradients, 4.2 GFLOPs
train: /home/c10/Рабочий стол/С10/C10M3/data/train... found 25000 images in 1000 classes ✅ 
val: /home/c10/Рабочий стол/С10/C10M3/data/val... found 2000 images in 1000 classes ✅ 
test: /home/c10/Рабочий стол/С10/C10M3/data/test... found 1000 images in 1000 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 63/63 [00:10<00:00,  6.14it/s]


                   all      0.002      0.006
Speed: 0.0ms preprocess, 3.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train6
Ultralytics 8.3.70 🚀 Python-3.11.2 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
YOLO11n-cls summary (fused): 112 layers, 2,807,024 parameters, 0 gradients, 4.2 GFLOPs
train: /home/c10/Рабочий стол/С10/C10M3/data/train... found 25000 images in 1000 classes ✅ 
val: /home/c10/Рабочий стол/С10/C10M3/data/val... found 2000 images in 1000 classes ✅ 
test: /home/c10/Рабочий стол/С10/C10M3/data/test... found 1000 images in 1000 classes ✅ 


val: Scanning /home/c10/Рабочий стол/С10/C10M3/data/val... 2000 images, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 125/125 [00:09<00:00, 12.88it/s]


                   all      0.002      0.006
Speed: 0.0ms preprocess, 3.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train62


NameError: name 'datetime' is not defined

In [15]:
model = YOLO("yolo11n-cls.pt")
model.train(data='./data/', epochs=1,verbose=False)
metrics = model.val()

Ultralytics 8.3.70 🚀 Python-3.11.2 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=./data/, epochs=1, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /home/c10/Рабочий стол/С10/C10M3/data/train... 25000 images, 0 corrupt: 100%|██████████| 25000/25000 [00:00<?, ?it/s]
val: Scanning /home/c10/Рабочий стол/С10/C10M3/data/val... 2000 images, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=1e-05, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 0 dataloader workers
Logging results to runs/classify/train5
Starting training for 1 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 63/63 [00:13<00:00,  4.83it/s]


                   all      0.002      0.006

1 epochs completed in 0.143 hours.
Optimizer stripped from runs/classify/train5/weights/last.pt, 5.8MB
Optimizer stripped from runs/classify/train5/weights/best.pt, 5.8MB

Validating runs/classify/train5/weights/best.pt...
Ultralytics 8.3.70 🚀 Python-3.11.2 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
YOLO11n-cls summary (fused): 112 layers, 2,807,024 parameters, 0 gradients, 4.2 GFLOPs
train: /home/c10/Рабочий стол/С10/C10M3/data/train... found 25000 images in 1000 classes ✅ 
val: /home/c10/Рабочий стол/С10/C10M3/data/val... found 2000 images in 1000 classes ✅ 
test: /home/c10/Рабочий стол/С10/C10M3/data/test... found 1000 images in 1000 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 63/63 [00:10<00:00,  6.10it/s]


                   all      0.002      0.006
Speed: 0.0ms preprocess, 3.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train5
Ultralytics 8.3.70 🚀 Python-3.11.2 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
YOLO11n-cls summary (fused): 112 layers, 2,807,024 parameters, 0 gradients, 4.2 GFLOPs
train: /home/c10/Рабочий стол/С10/C10M3/data/train... found 25000 images in 1000 classes ✅ 
val: /home/c10/Рабочий стол/С10/C10M3/data/val... found 2000 images in 1000 classes ✅ 
test: /home/c10/Рабочий стол/С10/C10M3/data/test... found 1000 images in 1000 classes ✅ 


val: Scanning /home/c10/Рабочий стол/С10/C10M3/data/val... 2000 images, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 125/125 [00:08<00:00, 13.96it/s]


                   all      0.002      0.006
Speed: 0.0ms preprocess, 3.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train52


In [ ]:
from sklearn.metrics import precision_score, f1_score, roc_auc_score
from typing import Dict

def calculate_metrics(preds, targets):
    # Вычисление метрик
    precision_score_val = precision_score(targets, preds, average='weighted')
    f1_score_val = f1_score(targets, preds, average='weighted')
    roc_auc_score_val = roc_auc_score(targets, preds)
    
    # Формирование словаря результатов
    metrics_dict = {
        'precision': precision_score_val,
        'f1': f1_score_val,
        'roc_auc': roc_auc_score_val
    }
    
    return metrics_dict